In [18]:
import os
import sys
import json

from typing import Any, Dict, List

In [99]:
def get_num_perfects_with_efficiency(episodes: List[Dict[str, Any]]):
    
    num_perfects = 0
    num_stopped = 0
    running_avg_efficiency = 0
    
    for i, episode in enumerate(episodes):
        called_stop = episode['called_stop']
        if called_stop:
            num_stopped += 1
            
            l1_distance = episode['l1_distance']
            if l1_distance == 0:
                num_perfects += 1

                actual = episode['elapsed_steps']
                
                init = episode['initial_rotation']
                target = episode['target_rotation']
                init_yaw = init['yaw']
                target_yaw = target['yaw']
                init_pitch = init['pitch']
                target_pitch = target['pitch']
                
                diff_pitch = abs(init_pitch - target_pitch)
                diff_yaw = abs(init_yaw - target_yaw)
                if diff_yaw > 180:
                    diff_yaw = 360 - diff_yaw
                
                efficiency = actual / (diff_pitch + diff_yaw + 1)

                running_avg_efficiency += efficiency
            
    running_avg_efficiency /= num_perfects
    num_perfects /= num_stopped
    
    return num_perfects, running_avg_efficiency

In [113]:
path = './benchmarks/fm_orb20.json'

In [114]:
with open(path, 'r') as f:
    data = json.load(f)

In [115]:
episodes = data['episodes_metrics']

In [116]:
get_num_perfects_with_efficiency(episodes)

(0.5, 1.992857142857143)

In [117]:
tmp_path = './benchmarks/classic/bounded_{diff}/fm_{method}{id}.json'

In [118]:
diffs = ['easy', 'medium', 'hard']
methods = ['orb', 'sift']
orb_ids = ['20', '30', '40', '60', '80', '100', '-nothresh']
sift_ids = ['20_small', '40', '50', '60', '80', '', '-nothresh']

In [120]:
for diff in diffs:
    print(diff)
    for method in methods:
        print(method)
        if method == 'orb':
            for orb_id in orb_ids:
                path = tmp_path.format(
                    diff=diff,
                    method=method,
                    id=orb_id,
                )
                if os.path.exists(path):
                    data = None
                    with open(path, 'r') as f:
                        data = json.load(f)
                    episodes = data['episodes_metrics']
                    print(orb_id, get_num_perfects_with_efficiency(episodes))
                else:
                    print(path, "doesn't exist!")
        elif method == 'sift':
            for sift_id in sift_ids:
                path = tmp_path.format(
                    diff=diff,
                    method=method,
                    id=sift_id,
                )
                if os.path.exists(path):
                    data = None
                    with open(path, 'r') as f:
                        data = json.load(f)
                    episodes = data['episodes_metrics']
                    print(sift_id, get_num_perfects_with_efficiency(episodes))
                else:
                    print(path, "doesn't exist!")
        print()

easy
orb
20 (0.4148471615720524, 1.2708784644790672)
30 (0.7648766328011611, 1.1505901352477417)
40 (0.8662790697674418, 1.121532480413315)
60 (0.9082969432314411, 1.1416222205981168)
80 (0.9097525473071325, 1.1389839103545598)
100 (0.9097525473071325, 1.1389839103545598)
-nothresh (0.9097525473071325, 1.1389839103545598)

sift
20_small (1.0, 4.305150250462882)
40 (0.9880774962742176, 1.6910555407228203)
50 (0.9897360703812317, 1.1975078605732734)
60 (0.9826338639652678, 1.1078810127025702)
80 (0.9855699855699855, 1.0540214394976302)
 (0.9927745664739884, 1.0469636530200508)
-nothresh (0.9971098265895953, 1.0005663298358871)

medium
orb
20 (0.3498542274052478, 1.6578300909533017)
30 (0.7161764705882353, 1.6453126243090797)
40 (0.8452380952380952, 1.5789919576657694)
60 (0.8695652173913043, 1.5816967133663489)
80 (0.8699551569506726, 1.5800282384437094)
100 (0.8699551569506726, 1.5800282384437094)
-nothresh (0.8699551569506726, 1.5800282384437094)

sift
20_small (0.9090909090909091, 2.8

In [107]:
tmp_path = './benchmarks/rl/bounded_{diff}/ppo_2_inverse_{id}.json'

In [108]:
ids = {
    'half': '30720000',
    'full': 'doublelengths2_59392000',
}

In [109]:
for diff in diffs:
    print(diff)
    for k, v in ids.items():
        path = tmp_path.format(
            diff=diff,
            id=v,
        )
        if os.path.exists(path):
            data = None
            with open(path, 'r') as f:
                data = json.load(f)
            episodes = data['episodes_metrics']
            print(k, get_num_perfects_with_efficiency(episodes))
        else:
            print(path, "doesn't exist!")

easy
half (0.9393939393939394, 2.0275906518557885)
full (0.9755747126436781, 1.865793602542363)
medium
half (0.9510086455331412, 1.88841928548768)
full (0.9826589595375722, 1.5974318131489063)
hard
half (0.9335260115606936, 1.7671257912520908)
full (0.9796806966618288, 1.4297348173502507)
